In [1]:
from dotenv import load_dotenv
import os

load_dotenv("D:/trading/.env")

api_key = os.getenv("TWELVE_API_KEY")
print("API Key cargada:", api_key[:6] + "..." if api_key else "No cargada")



API Key cargada: 0428b2...


In [2]:
import os
import shutil
from pathlib import Path
from dotenv import load_dotenv

# === CARGAR VARIABLES DESDE .env ===
load_dotenv("D:/trading/.env")
API_KEY = os.getenv("TWELVE_API_KEY")

if not API_KEY:
    raise ValueError("⚠️ No se pudo cargar la API key. Verificá el archivo .env")

# === CONFIGURACIÓN LOCAL ===
CONFIG_PATH = Path("D:/trading/config/450/symbol_groups.json")

# === Carpetas de salida ===
OUTPUT_DIR = Path("D:/trading/data/historic")
OUTPUT_RECENT_DIR = Path("D:/trading/data/historic_reciente")

# === LIMPIAR CARPETA DE SALIDA ===
if OUTPUT_DIR.exists():
    shutil.rmtree(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Carpeta {OUTPUT_DIR} limpiada y recreada.")

# Limpiar carpeta de históricos recientes
if OUTPUT_RECENT_DIR.exists():
    shutil.rmtree(OUTPUT_RECENT_DIR)
OUTPUT_RECENT_DIR.mkdir(parents=True, exist_ok=True)

print(f"🧹 Carpeta limpia: {OUTPUT_RECENT_DIR}")

✅ Carpeta D:\trading\data\historic limpiada y recreada.
🧹 Carpeta limpia: D:\trading\data\historic_reciente


In [3]:
import json
import time
import requests
import pandas as pd
from collections import deque

# === LECTURA DE SIMBOLOS ===
with open(CONFIG_PATH, "r") as f:
    grupos = json.load(f)

# === RATE LIMITER ===
class APIRateLimiter:
    def __init__(self, max_requests=5, window_seconds=60):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.timestamps = deque()

    def wait_if_needed(self):
        now = time.time()
        while self.timestamps and now - self.timestamps[0] > self.window_seconds:
            self.timestamps.popleft()
        if len(self.timestamps) >= self.max_requests:
            sleep_time = self.window_seconds - (now - self.timestamps[0]) + 0.5
            print(f"Esperando {sleep_time:.1f}s por rate limit...")
            time.sleep(sleep_time)
        self.timestamps.append(time.time())

rate_limiter = APIRateLimiter()


In [4]:
def fetch_data(symbol):
    rate_limiter.wait_if_needed()
    
    url = "https://api.twelvedata.com/time_series"
    params = {
        "symbol": symbol,
        "interval": "1day",
        "outputsize": 5000,
        "apikey": API_KEY,
        "format": "JSON"
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    if "values" not in data:
        raise ValueError(f"No data for {symbol}: {data.get('message', data)}")
    
    df = pd.DataFrame(data["values"])
    numeric_cols = ["open", "high", "low", "close", "volume"]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")
    df["datetime"] = pd.to_datetime(df["datetime"])
    df = df.sort_values("datetime")
    return df

def guardar_parquet(df, symbol):
    path = OUTPUT_DIR / f"{symbol}.parquet"
    df.to_parquet(path, index=False, compression="snappy")


In [5]:
total_ok = 0
total_error = 0

for grupo, simbolos in grupos.items():
    print(f"\n== Procesando {grupo} ({len(simbolos)} simbolos) ==")
    for i, symbol in enumerate(simbolos):
        try:
            df = fetch_data(symbol)
            guardar_parquet(df, symbol)
            print(f"{symbol}: OK")
            total_ok += 1
        except Exception as e:
            print(f"{symbol}: ERROR - {e}")
            total_error += 1
        if i < len(simbolos) - 1:
            time.sleep(11)

print(f"\nFinalizado: {total_ok} OK, {total_error} con error.")



== Procesando grupo_1 (5 simbolos) ==
A: OK
AAPL: OK
ABBV: OK
ACM: OK
ADBE: OK

== Procesando grupo_2 (5 simbolos) ==
Esperando 14.7s por rate limit...
ADI: OK
Esperando 0.8s por rate limit...
ADM: OK
ADSK: OK
AFG: OK
AFL: OK

== Procesando grupo_3 (5 simbolos) ==
Esperando 14.7s por rate limit...
AIG: OK
Esperando 0.8s por rate limit...
AKAM: OK
ALGN: OK
ALL: OK
ALLE: OK

== Procesando grupo_4 (5 simbolos) ==
Esperando 14.7s por rate limit...
ALLY: OK
Esperando 0.8s por rate limit...
ALNY: OK
AMAT: OK
AMD: OK
AME: OK

== Procesando grupo_5 (5 simbolos) ==
Esperando 14.6s por rate limit...
AMGN: OK
Esperando 0.8s por rate limit...
AMP: OK
AMT: OK
AMZN: OK
ANET: OK

== Procesando grupo_6 (5 simbolos) ==
Esperando 14.7s por rate limit...
ANSS: OK
Esperando 0.8s por rate limit...
APD: OK
APH: OK
APO: OK
ARCC: OK

== Procesando grupo_7 (5 simbolos) ==
Esperando 14.5s por rate limit...
ARE: OK
Esperando 0.7s por rate limit...
ARES: OK
ARMK: OK
ATI: OK
AVB: OK

== Procesando grupo_8 (5 simb

In [6]:
for archivo in OUTPUT_DIR.glob("*.parquet"):
    try:
        df = pd.read_parquet(archivo)

        # Convertir datetime → fecha si es necesario
        if "fecha" not in df.columns:
            if "datetime" in df.columns:
                df["fecha"] = pd.to_datetime(df["datetime"])
                df.drop(columns=["datetime"], inplace=True)
            else:
                print(f"⚠️ {archivo.name} no tiene ni 'fecha' ni 'datetime'. Saltado.")
                continue

        # Ordenar y limpiar duplicados
        df = df.sort_values("fecha").drop_duplicates("fecha")

        # Reescribir archivo completo (ordenado y limpio)
        df.to_parquet(archivo, index=False)

        # Crear y guardar recorte
        df_recent = df.tail(60)
        recent_path = OUTPUT_RECENT_DIR / archivo.name
        df_recent.to_parquet(recent_path, index=False)

        print(f"✅ {archivo.name}: actualizado y recorte generado.")

    except Exception as e:
        print(f"❌ Error procesando {archivo.name}: {e}")

✅ A.parquet: actualizado y recorte generado.
✅ AAPL.parquet: actualizado y recorte generado.
✅ ABBV.parquet: actualizado y recorte generado.
✅ ACM.parquet: actualizado y recorte generado.
✅ ADBE.parquet: actualizado y recorte generado.
✅ ADI.parquet: actualizado y recorte generado.
✅ ADM.parquet: actualizado y recorte generado.
✅ ADSK.parquet: actualizado y recorte generado.
✅ AFG.parquet: actualizado y recorte generado.
✅ AFL.parquet: actualizado y recorte generado.
✅ AIG.parquet: actualizado y recorte generado.
✅ AKAM.parquet: actualizado y recorte generado.
✅ ALGN.parquet: actualizado y recorte generado.
✅ ALL.parquet: actualizado y recorte generado.
✅ ALLE.parquet: actualizado y recorte generado.
✅ ALLY.parquet: actualizado y recorte generado.
✅ ALNY.parquet: actualizado y recorte generado.
✅ AMAT.parquet: actualizado y recorte generado.
✅ AMD.parquet: actualizado y recorte generado.
✅ AME.parquet: actualizado y recorte generado.
✅ AMGN.parquet: actualizado y recorte generado.
✅ AM